# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
data = pd.io.stata.read_stata('data/us-job-market-discrimination.dta')
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


First let us calculate the proportion of black and white applicants that received callbacks.

In [3]:
# applicants with black-sounding names
black = data[data.race=='b']

# applicants with white-sounding names
white = data[data.race=='w']

# proportion of callbacks for black applicants
Pblack = sum(black.call) / len(black)

# proportion of callbacks for white applicants
Pwhite = sum(white.call) / len(white)

In the sample, white applicants received 3.2% more callbacks than black applicants.

In [4]:
Pdiff = Pwhite - Pblack
print Pdiff

0.0320328542094


The null hypothesis is that race has no influence on the chance of getting a callback (Pwhite - Pblack = 0). The alternative hypothesis is that race does affect the chance of success. For the null hypothesis, the sample proportion is calculated with all the data regardless of race because we are assuming there is no difference between callbacks for black and white applicants.

In [5]:
n = len(data) / 2 # equal number of black and white applicants in the sample
Pnull = sum(data.call) / len(data)
sigma_null = np.sqrt(2 * Pnull * (1-Pnull) / n) # sigma for one group is sqrt(P * (1-P) / n)

The z-score for a 3.2% difference in the sample assuming there is no difference in the population is 4.1.

In [6]:
z = Pdiff / sigma_null # numerator only has observed difference because we assume no difference in the population
print z

4.10841215243


There is very little chance that there would be a 3.2% difference in the observed callback rates assuming the null hypothesis that race has no effect. When only accounting for race, there does appear to be a statistically significant difference between the callback rates for black and white applicants.

In [7]:
print norm.sf(z) * 2 # multiply by 2 for a two-tailed test

3.98388683759e-05


We are confident there is a 95% probability that the true difference between black and white callback rates is within 1.52% of the 3.2% observed in the sample.

In [8]:
sigma = np.sqrt(Pblack * (1-Pblack) / len(black) + Pwhite * (1-Pwhite) / len(white))
zcrit = norm.ppf(0.975) # 97.5% for a two-tailed 95% confidence interval
d = sigma * zcrit
print d

0.0152551260282


This yields a 95% confidence interval of 1.68% to 4.73%.

In [9]:
print (Pdiff - d, Pdiff + d)

(0.016777728181230755, 0.047287980237660412)


This can also be calculated with the interval method built into scipy.

In [10]:
print norm.interval(0.95, loc=Pdiff, scale=sigma)

(0.016777728181230755, 0.047287980237660412)


This analysis does not indicate that race is the most important factor in callback success because resume content was not included. Additional information on resume quality must be included to see if race has a meaningful impact on callback success for quality resumes. One could also break up the analysis by industry to see if certain industries are more biased than others.